In [1]:
import pandas as pd
import geopandas as gpd
#import hvplot.pandas  # noqa

#import cartopy.crs as ccrs

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
rental = pd.read_csv("govhack2021-main/python-app/rental_data.csv")

In [ ]:
aus = gpd.read_file("govhack2021-main/react-app/src/data/australia.json")
aus

In [ ]:
aus_postcode = gpd.read_file("govhack2021-main/react-app/src/data/australia_postcode.json")
aus_postcode

In [ ]:
post_to_sub = pd.read_excel("1270055006_CG_POSTCODE_2011_SA2_2011.xls", sheet_name="Table 3", skiprows=5)
post_to_sub.drop([0, 5989, 5990, 5991], axis=0, inplace=True)

In [ ]:
aus_postcode_sub = pd.merge(aus_postcode, post_to_sub, right_on='POSTCODE', left_on='POA_CODE')
aus_postcode_sub['POA_CODE'] = aus_postcode_sub['POA_CODE'].astype(int)

In [ ]:
aus_postcode_sub_rental = pd.merge(rental, aus_postcode_sub, left_on='Postcode', right_on='POA_CODE')
aus_postcode_sub_rental

In [ ]:
aus_postcode_sub_rental.drop(['RATIO', 'PERCENTAGE', 'POA_NAME', 'POSTCODE.1', 'POA_CODE'], axis=1, inplace=True)

In [ ]:
df = aus_postcode_sub_rental.copy()

In [ ]:
df['SA_NAME_2011_list'] = df.groupby(['Postcode', 'Year', 'Month'])['SA2_NAME_2011'].transform(lambda x: '-'.join(x))

In [ ]:
df = df[['Postcode', 'Year', 'Month', '# of Bonds', 'SQKM', 'geometry', 'SA_NAME_2011_list']].drop_duplicates()

In [ ]:
gdf = gpd.GeoDataFrame(df, geometry=df['geometry'])

In [ ]:
gdf.drop('Date', axis=1, inplace=True)

In [ ]:
gdf.to_file('australia_postcode_rentals1.json', driver="GeoJSON")